In [29]:
import requests

In [41]:
r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/update_candidates')

In [42]:
r.text

'candidates updated'

In [43]:
with PostgresDatabaseOperation() as cursor:
    sql= '''
    SELECT source_name,COUNT(article_id)
    FROM candidate_articles ca LEFT JOIN source_id_characteristics sc
    ON ca.source_id = sc.source_id
    GROUP BY source_name
    '''
    cursor.execute(sql)
    results = cursor.fetchall()

In [44]:
import pandas as pd

In [45]:
pd.DataFrame(results)

,0,1
0,None,421
1,navi,206
2,inc42,47
3,economic-times,13
4,investopedia,68
5,cnbc-tv-18,75
6,reuters,225
7,economist,9
8,live-mint,115
9,zerodha-varsity,34


In [46]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT article_id FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()

In [47]:
embedding_articles = [x[0] for x in results]

In [48]:
from sql.MongoDBArticle import MongoDBArticle

In [52]:
mongo_articles = MongoDBArticle.fetch_all_document_ids()

In [63]:
from bson import ObjectId

In [69]:
def fetch_documents_after_date(min_published_date):
    query = {"is_premium_article": False, "published_time": {"$gt": min_published_date}}
    projection = {"_id": 1}
    collection = MongoDBArticle.get_collection()
    documents = list(collection.find(query, projection))
    return [str(doc['_id']) for doc in documents]

In [53]:
len(mongo_articles)

235994

In [54]:
len(embedding_articles)

211131

In [55]:
excess_articles = set(embedding_articles) - set(mongo_articles)

In [57]:
len(excess_articles)

5290

In [58]:
missing_articles = set(mongo_articles) - set(embedding_articles)

In [59]:
len(missing_articles)

30153

In [67]:
latest_mongo_articles = fetch_documents_after_date('2024-01-01')

In [68]:
len(latest_mongo_articles)

36346

In [70]:
missing_latest_articles = set(latest_mongo_articles) - set(embedding_articles)

In [71]:
len(missing_latest_articles)

13747